In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch 
import os
import glob
from matplotlib import pyplot as plt
import os
import torch
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
from torch_models import  MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr
from torch_perturb.torch_pert_topk import PerturbedTopK
from distributions import QuantizedNormal



2024-08-22 13:56:07.931534: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 13:56:09.153530: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 13:56:09.153655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 13:56:09.445243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 13:56:09.851644: I tensorflow/core/platform/cpu_feature_guar

In [2]:
seed =360
num_components=2

In [3]:
perturbed_noise=0.01

In [4]:
# tracts/distributions
S=23

# total timepoints
T= 500
K=3

low_set_1_10 = [QuantizedNormal(10, 0.3) for _ in range(10)]
low_set_2_10 = [QuantizedNormal(30, 0.3) for _ in range(10)]
high_set_3 = [QuantizedNormal(50,3) for _ in range(3)]

dist_S = low_set_1_10 + low_set_2_10 + high_set_3

train_y_TS = np.zeros((T, S))
for s, dist in enumerate(dist_S):
    random_state = np.random.RandomState(10000 * seed + s*123456)
    train_y_TS[:, s] = dist.rvs(size=T, random_state=random_state)

model = MixtureOfTruncNormModel(num_components=num_components, S=S, low=0, high=100)

In [37]:
means = torch.tensor([1, 10])
softinv_means = means + torch.log(-torch.expm1(-means))
scales = torch.tensor([0.25, 0.25])
softinv_scales = scales - 0.2 + torch.log(-torch.expm1(-scales + 0.2)) 
#mix_weights = torch.log(1e-13 + torch.tensor(
#                                [[1,0]*20+[0,1]*3]))
mix_weights = torch.log(1e-13 + torch.tensor([[0.5, 0.5]*23]))
model.update_params(torch.cat([softinv_means, softinv_scales, mix_weights.view(-1)]))

In [38]:
step_size=0.01

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

In [40]:
M_score_func =  200
M_action = 200
train_T = train_y_TS.shape[0]
perturbed_top_K_func = PerturbedTopK(k=K, sigma=perturbed_noise)

In [50]:

optimizer.zero_grad()
mix_model = model()

y_sample_TMS = mix_model.sample((train_T, M_score_func))
y_sample_action_TMS = y_sample_TMS # AM I A DUMBASS???? mix_model.sample((train_T, M_action))

ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
ratio_rating_TS.requires_grad_(True)

def get_log_probs_baked(param):
    distribution = model.build_from_single_tensor(param)
    log_probs_TMS = distribution.log_prob(y_sample_TMS)

    return log_probs_TMS

jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, (model.params_to_single_tensor()), strategy='forward-mode', vectorize=True)

score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

# get gradient of negative bpr_t  with respect to ratio rating_TS
positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=K, perturbed_top_K_func=perturbed_top_K_func)
negative_bpr_loss = torch.mean(-positive_bpr_T)
loss=negative_bpr_loss

loss.backward()
loss_grad_TS = ratio_rating_TS.grad

gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
gradient_P = torch.sum(gradient_TSP, dim=[0,1])

gradient_tuple = model.single_tensor_to_params(gradient_P)

deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)
det_bpr =torch.mean(deterministic_bpr_T)
noisy_bpr = torch.mean(-negative_bpr_loss)

for param, gradient in zip(model.parameters(), gradient_tuple):
    param.grad = gradient
print(f'det bpr: {det_bpr}')
print(f'noisy bpr: {noisy_bpr}')

: 

In [47]:
optimizer.step()

In [48]:
gradient_P[-6:]

tensor([ 0.1883, -0.1883,  0.1856, -0.1856,  0.1834, -0.1834],
       grad_fn=<SliceBackward0>)

In [49]:
torch.softmax(model.mixture_probs[-3:], -1)

tensor([[0.4900, 0.5100],
        [0.4900, 0.5100],
        [0.4900, 0.5100]], grad_fn=<SoftmaxBackward0>)

In [ ]:
for param in model.parameters():
    print(param.grad)

In [17]:
# TODO: PRINT PARAMS

det bpr: 1.0


In [ ]:
# bash command to find the string "det bpr: 0.9" in a file in the directory
# grep -r "det bpr: 0.9" . 2>/dev/null